In [ ]:
import numpy.random
from kendall import Simulator, Entity, Stream, ExponentialSpawner, Queue, RandomSplitter, Splitter, Worker, Merger, Dropper
from kendall.main import SISOStream

def average(list):
    return sum(list) / float(len(list))

claims = set(["Claim A", "Claim B", "Claim C", "Claim D"])
deposits = set(["Deposit A", "Deposit B", "Deposit C", "Deposit D"])

def debug_entities(entity_list):
    print "Entity list: ", len(entity_list)
    for e in entity_list[:10]:
        print e.id
        for evt in e.event_list:
            if evt[2] in claims.union(deposits):
                print evt
        # print e.event_table

def time_between_events(entity_list, event1, event2):
    times = []
    for e in entity_list:
        if e.event_table.get(event1) and e.event_table.get(event2):
            time = e.event_table.get(event2) - e.event_table.get(event1)
            times.append(time)
    return times


class Guest(Entity):
    def __init__(self, *args, **kwargs):
        super(Guest, self).__init__(*args, **kwargs)
        self.seen_attractions = 0
        # TODO: Make the guest speed some random variable
        self.speed = 20

    def on_exit(self, stream, time):
        super(Guest, self).on_exit(stream, time)
        if stream.name in ["Attraction A", "Attraction B", "Attraction C", "Attraction D"]:
            self.seen_attractions += 1


class GuestSpawner(ExponentialSpawner):
    def create_entity(self, time):
        return Guest()

class DockDeposit(SISOStream):
    def __init__(self, *args, **kwargs):
        super(DockDeposit, self).__init__(*args, **kwargs)
        self.slots = kwargs.get("slots")
        self.free_slots = self.slots
        self.linked_claim = None

    def start(self):
        if self.linked_claim is None:
            raise Exception("DockDeposits need a linked DockClaim!!")
        self.free_slots = self.slots - self.linked_claim.bikes

    def enter(self, entity, time):
        # print "Guest {} entered {} with {} at {}".format(entity.id, self.name, self.free_slots, time)
        if self.free_slots > 0:
            self.free_slots -= 1
            self.linked_claim.put_bike(time)
            entity.on_enter(self, time)
            entity.on_exit(self, time)
            self.destination.enter(entity, time)
        else:
            raise Exception("You're not supposed to deposit a bike with no empty slots")

    def ready(self):
        return self.free_slots > 0 and self.destination.ready()

    def link_claim(self, claim):
        self.linked_claim = claim
        claim.linked_deposit = self

    def take_bike(self, time):
        self.free_slots += 1
        if self.free_slots == 1 and self.ready():
            self.source.notify_ready(self, time)

    # The next two methods are not necessary. I just put them for completeness
    def notify_ready(self, other, time):
        if self.ready():
            self.source.notify_ready(self, time)

    def notify_unready(self, other, time):
        if self.free_slots > 0:
            self.source.notify_unready(self, time)


class DockClaim(SISOStream):
    def __init__(self, *args, **kwargs):
        super(DockClaim, self).__init__(*args, **kwargs)
        self.bikes = kwargs.get("bikes")
        self.free_bikes = self.bikes
        self.linked_deposit = None

    def start(self):
        if self.linked_deposit is None:
            raise "DockClaims need a linked DockDeposit!!"
        self.free_bikes = self.bikes

    def enter(self, entity, time):
        # print "Guest {} entered {} with {} at {}".format(entity.id, self.name, self.free_bikes, time)
        if self.free_bikes > 0:
            self.free_bikes -= 1
            self.linked_deposit.take_bike(time)
            entity.on_enter(self, time)
            entity.on_exit(self, time)
            self.destination.enter(entity, time)
        else:
            raise "You're not supposed to claim a bike with no free bikes"

    def ready(self):
        return self.free_bikes > 0 and self.destination.ready()

    def put_bike(self, time):
        self.free_bikes += 1
        if self.free_bikes == 1 and self.ready():
            self.source.notify_ready(self, time)

    # The next two methods are not necessary. I just put them for completeness
    def notify_ready(self, other, time):
        if self.ready():
            self.source.notify_ready(self, time)

    def notify_unready(self, other, time):
        if self.free_slots > 0:
            self.source.notify_unready(self, time)


class ParkPath(Worker):
    def __init__(self, *args, **kwargs):
        super(ParkPath, self).__init__(*args, **kwargs)
        self.distance = float(kwargs["distance"])

    def time_to_finish(self, entity, time):
        return self.distance / entity.speed


class DroppingSplitter(Splitter):
    def start(self):
        super(DroppingSplitter, self).start()
        # Just need to remember who the dropper is for a minor speedup
        self.regulars = []
        self.droppers = []
        for d in self.destinations:
            if isinstance(d, Dropper):
                self.droppers.append(d)
            else:
                self.regulars.append(d)

    def select_destination(self, entity, time):
        if entity.seen_attractions >= 4:
            for d in self.ready_destinations:
                if d in self.droppers:
                    return d
        else:
            for d in self.destinations:
                if d in self.regulars:
                    return d

    
class ParkSimulator(Simulator):
    def __init__(self, *args, **kwargs):
        super(ParkSimulator, self).__init__(*args, **kwargs)
        self.dock_a_slots = kwargs.get("dock_a_slots", 80)
        self.dock_b_slots = kwargs.get("dock_b_slots", 80)
        self.dock_c_slots = kwargs.get("dock_c_slots", 80)
        self.dock_d_slots = kwargs.get("dock_d_slots", 80)
        self.dock_a_bikes = kwargs.get("dock_a_bikes", 80)
        self.dock_b_bikes = kwargs.get("dock_b_bikes", 80)
        self.dock_c_bikes = kwargs.get("dock_c_bikes", 80)
        self.dock_d_bikes = kwargs.get("dock_d_bikes", 80)
        Stream.simulator = self
        self.create_streams()

    def create_streams(self):
        self.spawner = GuestSpawner(name="Entrance", spawn_time=1.25)
        self.attraction_picker = RandomSplitter(name="Attraction Picker")
        self.dropper = Dropper(name="Exit")

        self.deposit_queue_a = Queue(name="Deposit Queue A", capacity=10**100)
        self.deposit_a = DockDeposit(name="Deposit A", slots=self.dock_a_slots)
        self.attraction_merger_a = Merger(name="Attraction Merger A")
        self.attraction_a = Worker(name="Attraction A", capacity=10**100)
        self.attraction_decision_a = DroppingSplitter(name="Attraction Decision A")
        self.claim_queue_a = Queue(name="Claim Queue A", capacity=10**100)
        self.claim_a = DockClaim(name="Claim A", bikes=self.dock_a_bikes)
        self.path_a_to_b = ParkPath(name="Path A to B", distance=3.0, capacity=10**100)
        self.deposit_a.link_claim(self.claim_a)

        self.deposit_queue_b = Queue(name="Deposit Queue B")
        self.deposit_b = DockDeposit(name="Deposit B", slots=self.dock_b_slots)
        self.attraction_merger_b = Merger(name="Attraction Merger B")
        self.attraction_b = Worker(name="Attraction B", capacity=10**100)
        self.attraction_decision_b = DroppingSplitter(name="Attraction Decision B")
        self.claim_queue_b = Queue(name="Claim Queue B", capacity=10**100)
        self.claim_b = DockClaim(name="Claim B", bikes=self.dock_b_bikes)
        self.path_b_to_c = ParkPath(name="Path B to C", distance=3.5, capacity=10**100)
        self.deposit_b.link_claim(self.claim_b)

        self.deposit_queue_c = Queue(name="Deposit Queue C")
        self.deposit_c = DockDeposit(name="Deposit C", slots=self.dock_c_slots)
        self.attraction_merger_c = Merger(name="Attraction Merger C")
        self.attraction_c = Worker(name="Attraction C", capacity=10**100)
        self.attraction_decision_c = DroppingSplitter(name="Attraction Decision C")
        self.claim_queue_c = Queue(name="Claim Queue C", capacity=10**100)
        self.claim_c = DockClaim(name="Claim C", bikes=self.dock_c_bikes)
        self.path_c_to_d = ParkPath(name="Path A to B", distance=4.0, capacity=10**100)
        self.deposit_c.link_claim(self.claim_c)

        self.deposit_queue_d = Queue(name="Deposit Queue D")
        self.deposit_d = DockDeposit(name="Deposit D", slots=self.dock_d_slots)
        self.attraction_merger_d = Merger(name="Attraction Merger D")
        self.attraction_d = Worker(name="Attraction D", capacity=10**100)
        self.attraction_decision_d = DroppingSplitter(name="Attraction Decision D")
        self.claim_queue_d = Queue(name="Claim Queue D", capacity=10**100)
        self.claim_d = DockClaim(name="Claim D", bikes=self.dock_d_bikes)
        self.path_d_to_a = ParkPath(name="Path D to A", distance=2.0, capacity=10**100)
        self.deposit_d.link_claim(self.claim_d)

        self.deposit_queue_a.pipe(self.deposit_a)
        self.deposit_a.pipe(self.attraction_merger_a)
        self.attraction_merger_a.pipe(self.attraction_a)
        self.attraction_a.pipe(self.attraction_decision_a)
        self.attraction_decision_a.pipe(self.claim_queue_a)
        self.claim_queue_a.pipe(self.claim_a)
        self.claim_a.pipe(self.path_a_to_b)
        self.path_a_to_b.pipe(self.deposit_queue_b)

        self.deposit_queue_b.pipe(self.deposit_b)
        self.deposit_b.pipe(self.attraction_merger_b)
        self.attraction_merger_b.pipe(self.attraction_b)
        self.attraction_b.pipe(self.attraction_decision_b)
        self.attraction_decision_b.pipe(self.claim_queue_b)
        self.claim_queue_b.pipe(self.claim_b)
        self.claim_b.pipe(self.path_b_to_c)
        self.path_b_to_c.pipe(self.deposit_queue_c)

        self.deposit_queue_c.pipe(self.deposit_c)
        self.deposit_c.pipe(self.attraction_merger_c)
        self.attraction_merger_c.pipe(self.attraction_c)
        self.attraction_c.pipe(self.attraction_decision_c)
        self.attraction_decision_c.pipe(self.claim_queue_c)
        self.claim_queue_c.pipe(self.claim_c)
        self.claim_c.pipe(self.path_c_to_d)
        self.path_c_to_d.pipe(self.deposit_queue_d)

        self.deposit_queue_d.pipe(self.deposit_d)
        self.deposit_d.pipe(self.attraction_merger_d)
        self.attraction_merger_d.pipe(self.attraction_d)
        self.attraction_d.pipe(self.attraction_decision_d)
        self.attraction_decision_d.pipe(self.claim_queue_d)
        self.claim_queue_d.pipe(self.claim_d)
        self.claim_d.pipe(self.path_d_to_a)
        self.path_d_to_a.pipe(self.deposit_queue_a)

        self.spawner.pipe(self.attraction_picker)

        self.attraction_picker.pipe(self.attraction_merger_a)
        self.attraction_picker.pipe(self.attraction_merger_b)
        self.attraction_picker.pipe(self.attraction_merger_c)
        self.attraction_picker.pipe(self.attraction_merger_d)

        self.attraction_decision_a.pipe(self.dropper)
        self.attraction_decision_b.pipe(self.dropper)
        self.attraction_decision_c.pipe(self.dropper)
        self.attraction_decision_d.pipe(self.dropper)

    def reset(self):
        super(ParkSimulator, self).reset()
        Entity.id = 0

    def analyze(self):
        # debug_entities(self.entity_list)
        pass

sim = ParkSimulator(time_limit=100)
for x in xrange(1, 100):
    processing_times = []
    sim.reset()
    sim.run()
    sim.analyze()
